# Black Friday Contest

In [40]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [41]:
df = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [43]:
df.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,376430.000000,166821.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,9.842329,12.668243,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,5.086590,4.125338,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,2.000000,3.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [44]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


Varificação se existem dados faltantes nas colunas do dataframe

In [45]:
df.isnull().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

Como podemos observar, as colunas Product_Category_2 e Product_Category_3 apresentam um grande volume de dados faltantes e, essa informação não pode ser descartado, por isso é preciso pensar em estratégias para essas colunas, uma delas seria dropar ambas, mas talvez esse não seria o melhor forma de lidar com a situação. Antes de tomar uma decisão, será feita uma verificação nesses dados para validar a melhor estratégia.

In [46]:
print(f"""Se optasse por remover as colunas com dados faltantes seriam retiradas um total de {colored(df.Product_Category_2.isnull().sum() + df.Product_Category_3.isnull().sum(), 'red')} de {colored(len(df), 'blue')} linhas no total presentes no dataframe estudado""")

Se optasse por remover as colunas com dados faltantes seriam retiradas um total de 556885 de 550068 linhas no total presentes no dataframe estudado


Como podemos ver seriam retiradas mais linhas do que existem no dataframe.

Agora podemos observar os dados das colunas e montar um nova estratégia.

In [47]:
display(
  df.Product_Category_2.unique(),
  df.Product_Category_3.unique()
)

array([nan,  6., 14.,  2.,  8., 15., 16., 11.,  5.,  3.,  4., 12.,  9.,
       10., 17., 13.,  7., 18.])

array([nan, 14., 17.,  5.,  4., 16., 15.,  8.,  9., 13.,  6., 12.,  3.,
       18., 11., 10.])

Existem algumas classes mas não existe nenhuma com 0, talvez seja uma boa estratégia utilizar essa classe como classe inválida para aqueles dados que não possuem dados, assim o dataframe seria totalmente preenchido e saberíamos que as classes 0 nessas colunas representariam linhas sem classe.

In [53]:
df.Product_Category_2.fillna(0, inplace=True)
df.Product_Category_3.fillna(0, inplace=True)
df.isnull().sum()

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
Purchase                      0
dtype: int64

In [49]:
df.dtypes

User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

As colunas que foram adicionadas 0 estam como float devido aos valores nulos que haviam previamente, como a coluna apresenta categorias, faria sentido passar esses dados para int

In [50]:
df.Product_Category_2 = df.Product_Category_2.astype('int64')
df.Product_Category_3 = df.Product_Category_3.astype('int64')

In [51]:
df.dtypes

User_ID                        int64
Product_ID                    object
Gender                        object
Age                           object
Occupation                     int64
City_Category                 object
Stay_In_Current_City_Years    object
Marital_Status                 int64
Product_Category_1             int64
Product_Category_2             int64
Product_Category_3             int64
Purchase                       int64
dtype: object

#### Análise dos campos User_ID e Product_ID

Agora vamos checar o as informações em relação aos produtos e usuários, primeiro verificando quantos usuários e quantos produtos temos únicos no dataframe.

In [56]:
print(f"Existem {colored(len(df.User_ID.unique()), 'yellow')} usuários únicos no dataframe e {colored(len(df.Product_ID.unique()), 'yellow')} produtos únicos no dataframe")

Existem 5891 usuários únicos no dataframe e 3631 produtos únicos no dataframe


#### Análise dos campos Gender, Age, Occupation, City_Category, Stay_In_Current_City_Years

Agora vamos observar as outras colunas e quais informações podemos retirar delas.
O comando head, info e dtype já nos deu um spoiler de como esses campos são e como foram representados.

In [59]:
for col_name in ['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years']:
    print(sorted(df[col_name].unique()))

['F', 'M']
['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
['A', 'B', 'C']
['0', '1', '2', '3', '4+']


O campo género é bináio e contém os sexos Feminino e Masculino representados em forma de caracter, já o campo de idades é do tipo string e apresenta um range de idades que o comprador se encaixa. 